<a href="https://colab.research.google.com/github/srikanthpurimitla/BDA/blob/main/ALMOST_DONE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx
import random
import docx
import os
import pandas as pd
import numpy as np

In [ ]:
def create_timetable_doc(title, timetable, filename):
    """Creates and saves a timetable document."""
    doc = docx.Document()
    doc.add_heading(title, 0)

    table = doc.add_table(rows=1, cols=7)
    table.style = "Table Grid"
    hdr_cells = table.rows[0].cells

    time_slots = ["7:30-8:20", "8:20-9:10", "9:10-10:00","10:30-11:20", "11:20-12:10", "12:10-1:00"]
    for i, slot in enumerate(time_slots):
        hdr_cells[i + 1].text = slot

    days = ["MON", "TUE", "WED", "THU", "FRI", "SAT"]
    for day_index, day in enumerate(days):
        row_cells = table.add_row().cells
        row_cells[0].text = day
        for period_index, period in enumerate(timetable[day_index]):
            if period:
                row_cells[period_index + 1].text = f"{period[0]} ({period[1]})"

    doc.save(filename)
    os.system(f"start {filename}")

In [ ]:
def assign_first_periods(tt, teachers, teacher_subject_index, teacher_limit_index):
    """Ensures each teacher gets at least one first-period slot, excluding HODs and Principals."""
    assigned_teachers = set()
    days = len(tt)

    for day in range(days):
        for teacher in teachers:
            if ("HOD" in teacher[0] or "Principal" in teacher[0]):  # Skip HODs and Principals
                continue
            if teacher[teacher_limit_index] > 0 and teacher[0] not in assigned_teachers:
                tt[day][0].extend([teacher[teacher_subject_index], teacher[0]])
                teacher[teacher_limit_index] -= 1
                assigned_teachers.add(teacher[0])
                break


In [ ]:
def can_assign_teacher(tt, teacher_name, subject, day, period):
    """Check if the teacher is already assigned at the same time in another class or consecutively in the same class."""
    # Check if the teacher is assigned in another class at the same time
    for ttable in [timetables]:
        if ttable[day][period] and ttable[day][period][1] == teacher_name:
            return False  # The teacher is already teaching in another class at this time

    # Check if the teacher is teaching the same subject consecutively in the same class
    if period > 0 and tt[day][period - 1] and tt[day][period - 1][1] == teacher_name:
        return False  # Teacher is teaching back-to-back periods

    # Check if the teacher already taught the same subject in this class today
    for p in range(6):  # Assuming 6 periods per day
        if tt[day][p] and tt[day][p][1] == teacher_name and tt[day][p][0] == subject:
            return False  # The teacher already taught this subject in this class today

    return True

In [ ]:
def arrange_timetable(tt, teacher_subject_index, teacher_limit_index, noofteachers, teachers):
    """Arranges a timetable ensuring teachers are assigned correctly based on constraints."""
    assign_first_periods(tt, teachers, teacher_subject_index, teacher_limit_index)

    for day in range(6):  # Iterate over days
        for period in range(6):  # Iterate over periods
            if not tt[day][period]:  # If the period is empty
                random.shuffle(teachers)  # Shuffle to ensure randomness
                for teacher in teachers:
                    if teacher[teacher_limit_index] > 0:
                        teacher_name = teacher[0]  # Get teacher's name
                        subject = teacher[teacher_subject_index]  # Get subject

                        if ("Principal" in teacher_name or "HOD" in teacher_name) and (period == 0 or period == 5):
                            continue  # Skip assigning to 1st and 6th period

                        if can_assign_teacher(tt, teacher_name, subject, day, period):
                            teacher[teacher_limit_index] -= 1
                            tt[day][period].extend([subject, teacher_name])
                            break  # Move to the next period


In [ ]:
#other dept staff detail
def other_dept_faculties(timetables,ch):
  temp=True
  while temp:
    sub=input('enter subject:')
    day=int(input('enter day choice\n1\n2\n3\n4\n5\n6'))
    period=int(input('enter period number:'))
    teacher=input('enter teacher name:')
    timetables[ch-1][day-1][period-1]=sub,teacher
    add_fac=int(input('want to add more \1.yes    2.no:'))
    if add_fac==2:
      temp=False

In [ ]:
#lab allocation method
def lab_all(timetables,ch,day,shift,lab,staff):
  if shift==1:
    timetables[ch-1][day-1][0]=lab+"LAB",staff
    timetables[ch-1][day-1][1]=lab+"LAB",staff
    timetables[ch-1][day-1][2]=lab+"LAB",staff
  elif shift==2:
    timetables[ch-1][day-1][3]=lab+"LAB",staff
    timetables[ch-1][day-1][4]=lab+"LAB",staff
    timetables[ch-1][day-1][5]=lab+"LAB",staff
  else:
    print('enter valid shift')

In [ ]:
#day and lab shifts
def day_shift(num_of_labs):
  for i in range(num_of_labs):
    day=int(input('enter day choice\n1\n2\n3\n4\n5\n6'))
    shift=int(input('enter shift choice\n1\n2'))
    lab=input('enter lab name:')
    staff=input('faculties:')
    lab_all(timetables,ch,day,shift,lab,staff)
  other_dept=int(input('if other dept faculties available\nenter number\n1.yes   2.no\n'))
  if other_dept==1:
    other_dept_faculties(timetables,ch)

In [ ]:
# Input Data
noofteachers = 25
'''
teachers = [["Mr.B.MANIKANTA","C","PYTHON",2,3,"102","",0,0,"","",3,0,"","PYTHON",2,3,"103","",0,0,"","",0,0,"","EITK",2,0,"","EH&SE",4,3,"602","EITK",2,0,"","",0,0,""],
["Mr.V.CHAKRADHAR HOD","C","",0,0,"","",0,0,"","OS",4,0,"","OS",4,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mrs. R. HIMABINDU","A","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","ML",4,3,"401","ML",4,3,"501","",0,0,"","",0,0,"","NLP",4,3,"701"],
["Mr. P.V. NAGA SRINIVAS","C","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","ATFL",4,0,"","ATFL",4,0,"","",0,0,""],
["Mr. D SEKHAR BABU" ,"B","",0,0,"","DS",4,3,"201","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr.T N SIVA KUMAR","C","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","EP",3,3,"404","EP",3,3,"503","",0,0,"","",0,0,"","",0,0,""],
["Mr.V. ROSHAN KUMAR","D","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr.MD.ISMAIL KHAN","A","CO",4,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","DHV",4,3,"702","",0,0,""],
["Mr.R. KARTHEEK","C","OOPS",4,3,"101","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","CCIDF",3,0,""],
["Mr.P. BHANU CHAND","C","",0,0,"","",0,0,"","DS",4,3,"301","",0,0,"","DS",4,3,201,"",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr.N. VENKATESWARA RAO","C","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","DMDW",4,3,"403","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mrs.D. DWIJA","A","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","SE",4,3,"402","",0,0,"","SE",4,3,"601","",0,0,"","",0,0,""],
["Mrs.K. NEELIMA" ,"C","",0,0,"","OOPS",4,3,"202","",0,0,"","",0,0,"","OOPS",4,3,"102","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mrs.  I BHAVANA","A","",0,0,"","CO",4,0,"","",4,3,"","CO",4,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr.K JOHN BUNYAN","A","",0,0,"","",0,0,"","",4,3,"","",0,0,"","",0,0,"","EITK",2,0,"","CN",3,0,"","EITK",2,0,"","CN",3,0,"","",0,0,""],
["Mrs. M. RISHITHA","B","",0,0,"","",0,0,"","",4,3,"","",0,0,"","",0,0,"","CN",3,0,"","",0,0,"","CN",3,0,"","",0,0,"","",0,0,""],
["Mrs. G.V. LEELAKUMARI","C","",0,0,"","",0,0,"","OOPS",4,3,"302","OOPS",4,3,"102","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["Mr. M BABU RAO","A","DS",4,3,"101","",0,0,"","",0,0,"","DS",4,3,"101","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","BDA",4,3,"702"],
["Mr.M.VEERABRAHMAM","C","",0,0,"","",0,0,"","PE&HV",2,0,"","",0,0,"","PE&HV",2,0,"","",0,0,"","SE",4,3,"501","",0,0,"","SE",4,3,"701","",0,0,""],
["Dr.P.RAMESH BABU","A","OS",4,0,"","",0,0,"","",0,0,"","",0,0,"","OS",4,0,"","",0,0,"","DMDW",4,3,"503","",0,0,"","",0,0,"","",0,0,""],
["Mr.B.KISHORE BABU","C","",0,0,"","",0,0,"","CO",4,0,"","",0,0,"","CO",4,0,"","",0,0,"","",0,0,"","IISCL",3,0,"","IISCL",3,0,"","",0,0,""],
["Ms.ALLU ANEESHA","A","",0,0,"","PYTHON",2,3,"201","PYTHON",2,3,"302","",0,0,"","PYTHON",2,3,"103","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["P. BHAVANI","B","PE&HV",2,0,"","PE&HV",2,0,"","",0,0,"","PE&HV",2,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["MATH","A","P&S",4,0,"","P&S",4,0,"","P&S",4,0,"","P&S",4,0,"","P&S",4,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,""],
["IMED","C","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","IMED",4,0,""],
["OE","B","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","",0,0,"","OE",4,0,""]]
'''
#file upload
import pandas as pd
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

teachers = pd.read_csv(fn)
teachers = teachers.fillna("")


rows = teachers.values.tolist()

# Timetables
#print(teachers)
timetables=[[[[], [], [], [], [], []] for _ in range(6)],[[[], [], [], [], [], []] for _ in range(6)],[[[], [], [], [], [], []] for _ in range(6)],[[[], [], [], [], [], []] for _ in range(6)],[[[], [], [], [], [], []] for _ in range(6)],[[[], [], [], [], [], []] for _ in range(6)],[[[], [], [], [], [], []] for _ in range(6)],[[[], [], [], [], [], []] for _ in range(6)],[[[], [], [], [], [], []] for _ in range(6)],[[[], [], [], [], [], []] for _ in range(6)]]


# Arrange Timetables

j=2
for i in range(10):
  arrange_timetable(timetables[i], j, j+1, noofteachers, rows)
  j+=4
  #lab allocation
print('lab allocation')
num_of_class=int(input('enter number of classes upto 10 classes:'))
#classes
classes = [
    "2/4 AIML-A",
    "2/4 AIML-B",
    "2/4 CB",
    "2/4 DS-A",
    "2/4 DS-B",
    "3/4 AIML-A",
    "3/4 AIML-B",
    "3/4 CB",
    "3/4 DS",
    "4/4 CBDS"
]
while True:
  print('enter the class choice:')

  for i in range(num_of_class):
    print(f'class {classes[i]}'':',i+1)
  print('if you want exit click :99')
  ch=int(input())
  if ch==1:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==2:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==3:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==4:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==5:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==6:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==7:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==8:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==9:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==10:
    num_of_labs=int(input('enter number of labs:'))
    day_shift(num_of_labs)
  elif ch==99:
    break
  else:
    print('enter valid class')


# Create Word Documents

for i in range(10):
  create_timetable_doc(classes[i], timetables[i], f"{i+1}.docx")


Saving project_main.csv to project_main (13).csv
User uploaded file "project_main (13).csv" with length 3069 bytes
lab allocation
enter number of classes upto 10 classes:10
enter the class choice:
class 2/4 AIML-A: 1
class 2/4 AIML-B: 2
class 2/4 CB: 3
class 2/4 DS-A: 4
class 2/4 DS-B: 5
class 3/4 AIML-A: 6
class 3/4 AIML-B: 7
class 3/4 CB: 8
class 3/4 DS: 9
class 4/4 CBDS: 10
if you want exit click :99
1
enter number of labs:1
enter day choice
1
2
3
4
5
61
enter shift choice
1
21
enter lab name:PYTHON
faculties:SK AND GK
if other dept faculties available
enter number
1.yes   2.no
1
enter subject:FUCK U
enter day choice
1
2
3
4
5
66
enter period number:3
enter teacher name:PIG
want to add more .yes    2.no:2
enter the class choice:
class 2/4 AIML-A: 1
class 2/4 AIML-B: 2
class 2/4 CB: 3
class 2/4 DS-A: 4
class 2/4 DS-B: 5
class 3/4 AIML-A: 6
class 3/4 AIML-B: 7
class 3/4 CB: 8
class 3/4 DS: 9
class 4/4 CBDS: 10
if you want exit click :99
99
